## Installation and deployment of 5G System using the Open Air Interface (OAI)
## Objectives
- Install and Deploy 5GC, gNB and UE using OAI wiht Docker



## 5GC Installation

In a Linux Terminal, follow the steps below:

**Step 0:** Setup your VM or Baremetal Machine, after install Useful Software:

sudo su

apt update -y

apt upgrade -y

apt install -y python3-pip git curl cmake tree golang vim net-tools build-essential nfs-common openssh-server

**Step 1:** Install dependencies

sudo apt-get install ca-certificates 

sudo apt-get install curl 

sudo apt-get install gnupg 

sudo apt-get install lsb-release 

**Step 2:** Enable packet forwarding

sudo sysctl net.ipv4.conf.all.forwarding=1

sudo iptables -P FORWARD ACCEPT

**Step 3:** Install docker and docker-compose

sudo rm /etc/apt/sources.list.d/docker.list*

sudo mkdir -p /etc/apt/keyrings 

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu \\$(. /etc/os-release; echo "\\$UBUNTU_CODENAME") stable"

sudo chmod a+r /etc/apt/keyrings/docker.gpg

sudo apt-get update

sudo apt-get install -y docker-ce docker-ce-cli containerd.io docker-compose-plugin

```python 
sudo curl -L https://github.com/docker/compose/releases/download/1.29.2/docker-compose-`uname -s`-`uname -m` -o /usr/local/bin/docker-compose
```

sudo chmod +x /usr/local/bin/docker-compose

**Step 4:** Pull the images from Docker Hub

docker pull mysql:8.0

docker pull oaisoftwarealliance/oai-amf:v1.5.0

docker pull oaisoftwarealliance/oai-nrf:v1.5.0

docker pull oaisoftwarealliance/oai-smf:v1.5.0

docker pull oaisoftwarealliance/oai-spgwu-tiny:v1.5.0

docker pull oaisoftwarealliance/trf-gen-cn5g:focal

docker pull oaisoftwarealliance/oai-gnb:develop

docker pull oaisoftwarealliance/oai-nr-ue:develop


**Step 5:** Clone OpenAirInterface 5G Core repository

git clone https://gitlab.eurecom.fr/oai/openairinterface5g.git

cd ci-scripts/yaml_files/5g_rfsimulator


**Step 6:** Install and deploy 5GC:

docker-compose up -d mysql oai-nrf oai-amf oai-smf oai-spgwu oai-ext-dn

**Step 7:** Wait for a bit and test using:

docker-compose ps -a

You will see
     
![5GC_deploy_02](./FIGS/H01_FAST_UNI_III/img_01.png)

**Step 8:** Take care, to execute this step, the whole 5GC SHALL be in healthy state (especially the mysql container). Deploy monolithic gNB:

docker-compose up -d oai-gnb

**Step 9:** Wait for a bit and test using:

docker-compose ps -a

You will see
     
![5GC_deploy_02](./FIGS/H01_FAST_UNI_III/img_02.png)


**Step 10:** You can verify that the gNB is connected with the AMF:

docker logs rfsim5g-oai-amf

![5GC_deploy_02](./FIGS/H01_FAST_UNI_III/img_03.png)

**Step 11:** Deploy OAI NR-UE in RF simulator mode and in Standalone Mode

docker-compose up -d oai-nr-ue


**Step 10:** Wait for a bit and test using:

docker-compose ps -a

![5GC_deploy_02](./FIGS/H01_FAST_UNI_III/img_04.png)


**Step 13:** You can verify that the gNB is connected with the AMF:

docker logs rfsim5g-oai-amf


**Step 14:** Making sure the OAI UE is connected

docker exec -it rfsim5g-oai-nr-ue /bin/bash

Inside docker type: ifconfig

You will see:

![5GC_deploy_02](./FIGS/H01_FAST_UNI_III/img_05.png)

Type **exit** to return outside the UE pod.

**Step 15:** Deploy Second OAI NR-UE in RF simulator mode and in Standalone Mode

docker-compose up -d oai-nr-ue2

**Step 16:** start a iperf server inside U1

docker exec -it rfsim5g-oai-nr-ue1 /bin/bash

Inside docker type: iperf -B 12.1.1.2 -u -i 1 -s

You will see:

![5GC_deploy_02](./FIGS/H01_FAST_UNI_III/img_06.png)

**Step 16:** open a new terminal and start a iperf client inside UE2

docker exec -it rfsim5g-oai-nr-ue /bin/bash

Inside docker type: iperf -c 12.1.1.2 -u -i 1 -t 20 -b 500K

You will see:

![5GC_deploy_02](./FIGS/H01_FAST_UNI_III/img_07.png)


**Step 17:** to undeploy everything

docker-compose down



**Source:** https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/ci-scripts/yaml_files/5g_rfsimulator